## Lesson 13 — Concepts of clean coding

So far, we have learned mostly about how use elements of the language in a technical way. Today, we will cover what is expected of a good software developer, and what you should be focusing on when developing software.

Note that _"clean coding"_ is a very broad term, and everyone has a different concept on what clean code looks like. The original term comes from the book _Clean code_ by Robert C. Martin.

Readings:

* [_SOLID Design Principles Explained: Building Better Software Architecture_, by DigitalOcean](https://www.digitalocean.com/community/conceptual-articles/s-o-l-i-d-the-first-five-principles-of-object-oriented-design)
* [_Code Style_, by The Hitchhiker's guide to Python](https://docs.python-guide.org/writing/style/)
* [_Documentation_, by The Hitchhiker's guide to Python](https://docs.python-guide.org/writing/documentation/)
* [_Type hints cheat sheet_, by mypy documentation](https://mypy.readthedocs.io/en/stable/cheat_sheet_py3.html)
* [_What is CI/CD_, by Gitlab](https://about.gitlab.com/topics/ci-cd/)

Topics covered:

* [Introduction](#Introduction)
* [Code formatting](#Code-formatting)
* [Documentation styles](#Documentation-styles)
* [Type system](#Type-system)
* [KISS principle](#KISS-principle)
* [Automated testing](#Automated-testing)
* [Development workflow](#Development-workflow)
* [CI/CD](#CI/CD)

## Introduction

I'll only focus on the [SOLID principles](https://en.wikipedia.org/wiki/SOLID), then directly go into how to apply it into the Python language with examples.

_SOLID_ is an acronym, and stands for:

### S: Single-Responsibility Principle

> A class should have one and only one reason to change, meaning that a class should have only one job.

Avoid creating structures that have too many responsibilities. A class represents one thing, a function does one thing.

**Example**: if you want to create a class called `User`, the property `has_permission` would be included, but you **wouldn't** create a method called `log_in_and_check_permissions`, since the method does more than one thing.

### O: Open-Closed Principle

> Objects or entities should be open for extension but closed for modification.

Once an object is created (e.g., a class), you should treat this as an unmodifiable object, and assume people already depend on its interface.

The class can be extended, but current functionality remains the same.

**Example:** if you have a class `Rectangle` that takes `(x, y)` and returns the `area()`, it is correct to add an additional method called `perimeter()`, thus extending the class. You **cannot** rename `area()` to `ground()` or `section()`, since this method is considered unmodifiable.

### L: Liskov Substitution Principle

> Let q(x) be a property provable about objects of x of type T. Then q(y) should be provable for objects y of type S where S is a subtype of T.

This deals with inheritance, and essentially tells that classes and sub-classes are interchangeable.

**Example:** if you have a class called `Vehicle`, with the method `start`, then you create a sub-class `Car` and `Bus`, both with defined `start` methods, this would follow the principle. If I create a class called `Bike`, this **breaks the principle** since you cannot start a bike. You could, however, create an `EBike` class, that is also startable.

### I: Interface Segregation Principle

> A client should never be forced to implement an interface that it doesn’t use, or clients shouldn’t be forced to depend on methods they do not use.

In programming, interfaces represent ways to interact with a certain object, identifying which functions this interface offers you. The client should not be forced to implement an interface it does not use.

**Example**: in the previous example, we could get around the Liskov principle by implementing an empty `start` method for the `Bike`. Since we can consider the `Vehicle` as describing and interface (how to interact with _any_ vehicle), and we are _forced_ to implement something we do not use, this principle is broken.

### D: Dependency Inversion Principle

> Entities must depend on abstractions, not on concretions. It states that the high-level module must not depend on the low-level module, but they should depend on abstractions.

This is a bit more conceptual and requires you think of the high-level abstractions first, then proceed into the implementation.

This goes together with the interface principle, as it encourages of thinking of an interface first, then on the individual concrete components.

**Example:** If you implement a `USBDevice` (computer USB interface), this device will contain the `connect`, `send` and `receive` methods. This interface is entirely abstract, but you can create a _concrete_ implementation by extending this interface with for example a `Mouse` or `Keyboard`, that will extend the high-level model.

## Code formatting

### PEP8

One of the first proposals bringing standardization to the Python programming language was the [PEP8](https://peps.python.org/pep-0008/) (PEP stands for Python Enhancement Proposals).

It defined some basic standards on how Python projects should look like, to keep codebases consistent across the board, with the goal of keeping projects readable (one of the biggest focuses of the Python programming language). It's important to note that this only **provides advice**, and is not binding to your code.

Among other things, it defines:

- That code should be indented with **spaces** instead of **tabs**, and the indentation size should be **4 spaces**.

```python
# Correct:
def foobar():
    return None  # <- line starts with 4 spaces
```

- Imports should be done in separate lines.

```python
# Correct:
import os
import sys
# Wrong:
import sys, os
# It’s okay to say this though:
# Correct:
from subprocess import Popen, PIPE
```

- How to correctly space entries in statements or expressions.

```python
# Correct:
spam(ham[1], {eggs: 2})
# Wrong:
spam( ham[ 1 ], { eggs: 2 } )
```

- Use inline comments sparingly.

```python
# Avoid unnecessary comments
x = x + 1                 # Increment x
# Keep them when they have context that needs explaining
x = x + 1                 # Compensate for border
```

- Defines naming styles for classes and functions.

```python

TOTAL = 123   # <- Constants are written in all capital letters


class FooBar:  # <- Class names defined as CapWords convention

    # Always use self for the first argument to instance methods.
    def foo(self) -> None:   # <- Function names are defined in snake_case.
        return None
```

### Auto-formatters

You are of course **not expected to remember any of this styling**, that is why we have **auto-formatters**. They are the ones that will keep your code readable and consistent against the most up-to-date specifications (not only PEP8).

Since Python is an interpreted language, some formatters will also offer the option to lint your code, looking for possible anti-patterns in your code.

The current most used formatters are [black](https://github.com/psf/black), that defines the entire spec, including all formatting rules, and [ruff](https://docs.astral.sh/ruff/), that will apply black plus other linters.

You can try the [online version of black](https://black.vercel.app) to check how it formats the code.

I'll not mention other alternatives like [flake8](https://flake8.pycqa.org/en/latest/), which implement the Python PEP guidelines, but linters like ruff are preferred because it is able to handle larger codebases faster.

Here is an example of code before formatting:

```python
def very_important_function(template : str, * variables, file : os.PathLike, debug: bool = False, ):
    """Applies `variables` to the `template` and writes to `file`."""
    with open(
        file, "w"
    ) as f:
        ...
```

And after formatting:

```python
def very_important_function(
    template: str,
    *variables,
    file: os.PathLike,
    debug: bool = False,
):
    """Applies `variables` to the `template` and writes to `file`."""
    with open(file, "w") as f:
        ...
```

### Linters

One of the most common linters is [ruff](https://docs.astral.sh/ruff/). I'll display an example of why this linter is useful.

Take the following function:

```python
def function(argument: str) -> str:
    if argument == None:
        return "Hello World"
    return "Hello " + argument
```

While this expression looks fine at first, Python forbids the comparison to `NoneType`. If we run ruff against your file, it will tell us:

```bash
ruff check examples/ruff.py
```

Results in:

```python
E711 Comparison to `None` should be `cond is None`
 --> examples/ruff.py:3:20
  |
2 | def Function(argument: str):
3 |     if argument == None:
  |                    ^^^^
4 |         return "Hello World"
5 |     return "Hello " + argument
  |
help: Replace with `cond is None`

Found 1 error.
No fixes available (1 hidden fix can be enabled with the `--unsafe-fixes` option).
```

If we replace the condition with `argument is None`, the error goes away.

The linter will also detect bugs in your code that would cause it to crash. For example, when a variable is not properly defined, it will also raise a warning:

```python
def square():
    return n ** 2  # <- n wasn't declared in this scope
```

If I run the linter, it will display:

```python
F821 Undefined name `n`
 --> examples/ruff.py:8:12
  |
7 | def double():
8 |     return n * 2
  |            ^
  |

Found 1 error.
```

Note that you can get these warnings at lint time, rather than on runtime.

Sometimes, the errors are really just bad practices. See for example when we use a named lambda function:

```python
a = lambda: "Hello World"
```

We get:

```python
E731 Do not assign a `lambda` expression, use a `def`
 --> examples/ruff.py:7:1
  |
7 | a = lambda: "Hello World"
  | ^^^^^^^^^^^^^^^^^^^^^^^^^
  |
help: Rewrite `a` as a `def`

Found 1 error.
No fixes available (1 hidden fix can be enabled with the `--unsafe-fixes` option).
```

We can often fix such cases automatically, although I would recommend to fix them case-by-case. If we run the command `ruff check --unsafe-fixes --fix examples/ruff.py`, we get instead:

```python
def a():
    return "Hello World"
```

### Pre-commit integration

You can integrate automatic code formatting and linting directly on the git client using [pre-commit](https://pre-commit.com/), which is nowadays the recommended best-practice to keep the code formatting consistent. To do so, you can create a configuration file `.pre-commit-config.yaml` that contains the hooks that should run every time a commit is done.

pre-commit ensures that you files changed are automatically formatted every time you commit them to the git history. You can include multiple hooks in your pre-commit configuration:

<center><img src="../images/pre-commit.png"/></center>

When using pre-commit, if your code does not conform to the standards, it will not be possible to do the commit.

## Documentation styles

There are multiple standards of providing documentation as a docstring, most of them offering the same basic functionality.

The goal of the style is enforce how the documentation for each parameter and return type is stored on the docstring. Most IDEs will support displaying this documentation on hover. More advanced documentation standards also provide fields for defining examples or adding additional notes.

There are three main ways of providing documentation: _Sphinx_, _Google style_ and _numpydoc_.

**Note:** If you don't know which style to stick to, use Sphinx.

### Sphinx (reStructuredText)

Sphinx is the most popular documentation style. It is recommended for most projects as it contains the bare minimal that needs to be documentated about a function. A short and long descriptions, the description for the individual parameters used, as well as possible exceptions it can raise.

```python
def add_sphinx(a: int, b: int) -> int:
    """
    Add up two integer numbers.

    Long description of the method.

    :param a: First number to add.
    :param b: Second number to add.
    :returns:  The sum of `a` and `b`.
    :raises TypeError: If the types are passed incorrectly.
    """
    pass
```

### Google-style

Google-style documentation exists because Google has their [own public Python best-practices styleguide](https://google.github.io/styleguide/pyguide.html), and it also includes how to generate documentation.

```python
def add_google(a: int, b: int) -> int:
    """
    Add up two integer numbers.

    Long description of the method.

    Args:
        a (int) : First number to add.
        b (int) : Second number to add.

    Returns:
        The sum of `a` and `b`.

    Raises:
        TypeError: If the types are passed incorrectly.
    """
    pass
```

### Numpydoc

Numpy documentation is based on the Google format and includes some nice to haves when methods have a long list of parameters and/or the function documentation must be well structured. 

Note that it contains fields for the common things you would need for a mathematical function like references for citing articles, notes for describing certain behaviour, as well as examples on how to use the function properly.

Numpy has the special case that their libraries are used by a wide range of developers with different backgrounds, and they focus on quality documentation with references and different usages for their functions.

```python
def add_numpy(a: int, b: int) -> int:
    r"""
    Add up two integer numbers.

    Long description of the method.

    Parameters
    ----------
    a : int
        First number to add.
    b : int
        Second number to add.

    Returns
    -------
    int
        The sum of `a` and `b`.

    Raises
    ------
    TypeError
        If the types are passed incorrectly.

    See Also
    --------
    ...

    Notes
    -----
    ...

    References
    ----------
    ...

    Examples
    --------
    These are written in doctest format, and should illustrate how to
    use the function.

    >>> add_numpy(1, 2)
    3
    """
    pass
```

## Type system

The type system is there. Use it.

Look at the following example:

```python
def add(a, b):
    return a + b

add(1, 2)         # Results in 3
add("1", "2")     # Results in "12"
add(True, False)  # Results in 1?
add("1", 1)       # Results in TypeError: can only concatenate str (not "int") to str
```

To avoid those pitfalls when developing code, you should type it, and in best cases scenarios validate the type system using a static-type checker like [mypy](https://mypy-lang.org/). Let's now type our file

```python
def add(a: int, b: int) -> int:
    return a + b

add(1, 2)
add("1", "2")
add(True, False)
add("1", 1)
```

And run mypy on it:

```
mypy examples/type.py
```

We get:

```bash
examples/type.py:5: error: Argument 1 to "add" has incompatible type "str"; expected "int"  [arg-type]
examples/type.py:5: error: Argument 2 to "add" has incompatible type "str"; expected "int"  [arg-type]
examples/type.py:7: error: Argument 1 to "add" has incompatible type "str"; expected "int"  [arg-type]
Found 3 errors in 1 file (checked 1 source file)
```

We have now made sure that `add("1", "2")` and `add("1", 1)` are not accepted anymore in our function, the later causing an error in the application.

The line `add(True, False)` is still accepted by mypy as correct, since bool is a subclass from int.

**Note:** having a static-type checker will reduce development speed simply because it is trickier to figure out the types in complex scenarios (e.g., when dealing with coroutines or decorators). You should consider the use case-by-case.

### Prefer to return objects, not dictionaries

Since Python does not enforce its structure, you are free to make any design decisions for your argument and return types. Look at the following example:

```python
def parse_page(page: dict) -> dict:
    try:
        return {
            "metadata": page["page"].decode(),
            "page_num": page["page_num"],
        }
    except Exception as e:
        return {"error": f"Error processing page: {str(e)}"}
```

Even though we have typed the methods (with `dict`), this format still has disadvantages:

- Unclear how the `page` has to be provided.
- Unclear what the return type looks like.

This is because `dict` type is very flexible. We want to provide clarity on this method, so let's use the previously learned dataclasses to improve our typing:

```python
from dataclasses import dataclass


@dataclass
class Page:
    page: bytes
    page_num: int


@dataclass
class ParsedPage:
    metadata: str
    page_num: int



def parse_page(page: Page) -> ParsedPage | None:
    try:
        return ParsedPage(page.page.decode(), page.page_num)
    except Exception as e:
        return None
```

Now, we are much clearer on what we take as input, and what is output by the function.

### Avoid returning multiple types

Let's look at the previous function again, since there is another crucial mistake on the implementation:

```python
def parse_page(page: dict) -> dict:
    try:
        return {
            "metadata": page["page"].decode(),
            "page_num": page["page_num"],
        }
    except Exception as e:
        return {"error": f"Error processing page: {str(e)}"}


parsed_page = parse_page({})
print(parsed_page["metadata"])  # Returns an KeyError: 'metadata'
```

Note how the this simple code causes an error, that **does not** get spotted by either mypy or ruff linters. This is because our code returns multiple "types". In this case, there is only one type `dict`, but we can see them as _multiple_ since their format is different.

On the previous example, we have typed the `ParsedPage` and returned `ParsedPage | None` instead. The general guidelines applies:

- Return one type from your function.
- Sometimes, an optional `None` might be returned. This indicates missing value of treated error.
- If the response is not optional, raise an exception.

## KISS principle

The [KISS principle](https://en.wikipedia.org/wiki/KISS_principle) ("Keep It Simple, Stupid") is not really a guideline, just a reminder not to over complicate our designs prematurely.

<center><img src="https://imgs.xkcd.com/comics/the_general_problem.png"/></center>

One [example of this](https://docs.python-guide.org/writing/style/#explicit-code) is when making a function, opting to use the most straight forward way to do it:

**Bad**: untyped, overly complicated, non-specific on the argument types.

```python
def make_complex(*args):
    x, y = args
    return dict(**locals())
```

**Good**:

```python
def make_complex(x: int, y: int) -> dict:
    return {'x': x, 'y': y}
```

### Avoid premature optimization

Quoting Donald Knuth (emphasis mine):

> Programmers waste enormous amounts of time thinking about, or worrying about, the speed of noncritical parts of their programs, and these attempts at efficiency actually have a strong negative impact when debugging and maintenance are considered.
> We should **forget about small efficiencies**, say about 97% of the time: **premature optimization is the root of all evil**. Yet we should not pass up our opportunities in that critical 3%.

Here is a quite common example. We have a function that selects a sub-function based on the passed arguments.

```python
def function_if(v: int, param: str) -> int:
    if param == "a":
        return fn(v)
    elif param == "b":
        return fn(v)
    elif param == "c":
        return fn(v)
```

Let's try to "optimize" it early by using a dictionary instead:

```python
def function_dict(v: int, param: str) -> int:
    methods = {
        "a": fn,
        "b": fn,
        "c": fn,
    }
    return methods[param](v)
```

We effectively made our method less readable, without any clear benefit. This is because optimization:

- can only be done using a benchmark reference - we first need to figure out if our code is even a bottleneck.
- can only be done comparatively - is my new code now faster?
- if you sacrifice readability, there needs to be a clear reason, and this should be documented.
- Ideally, you should also have unit testing when optimizing a method, to ensure functionality has not changed.

Let's do it right and compare the two versions using the Python interpreter:

In [1]:
import time

In [7]:
# Define a dummy fn
def fn(x: int) -> int:
    return x

# Define the original function
def function_if(v: int, param: str) -> int:
    if param == "a":
        return fn(v)
    elif param == "b":
        return fn(v)
    elif param == "c":
        return fn(v)


# Define the "optimized" function
def function_dict(v: int, param: str) -> int:
    methods = {
        "a": fn,
        "b": fn,
        "c": fn,
    }
    return methods[param](v)


for function in (function_if, function_dict):
    t = time.perf_counter()
    for _ in range(1_000_000):
        function(1, "c")
    t_elapsed = time.perf_counter() - t
    print(f"Timing of {function.__name__}: {t_elapsed:.1f}s")

Timing of function_if: 0.1s
Timing of function_dict: 0.2s


Now we can conclude for sure: our new "optimized" function is clearly slower, and now we can decide next steps based on this information.

Python offers [profilers](https://docs.python.org/3/library/profile.html) for bottleneck identification, and those can be integrated into your IDE for performance improvements.

### Learn the language tools

Avoid rewriting things that are already supported by the language. If you come from languages like C, you might be familiar with the following pattern:

In [3]:
alphabet = ["a", "b", "c", "d", "e"]

In [4]:
i = 0
while i < 5:
    print(i, alphabet[i])
    i = i + 1

0 a
1 b
2 c
3 d
4 e


Python supports for loops, so we can improve this loop to turn it a bit more _"Pythonic"_:

In [5]:
for i in range(5):
    print(i, alphabet[i])

0 a
1 b
2 c
3 d
4 e


If you are more experienced, you will notice that the tools to iterate over objects are readily available to make your code cleaner and easier to read:

In [6]:
for i, letter in enumerate(alphabet):
    print(i, letter)

0 a
1 b
2 c
3 d
4 e


### Avoid nesting

Nesting can make your code very difficult to understand. This is because each instance of nesting overloads your brain with +1 possibility that _could_ happen if that condition is triggered.

Let's look at the following example that retrieves an user from the database:

```python
def get_user_from_database(user_id: str) -> str | None:
    if user_id is not None:
        return User(user_id)
    else:
        return None


def function(user_id: str) -> User:
    user = get_user_from_database(user_id)
    if user:
        if user.has_permissions():
            return user
        else:
            raise ValueError("user does not have permissions")
    else:
        raise ValueError("User not found")
```

Here, your brain has to go up and down multiple times to read the method and figure out what is going on.

There is a good trick to reduce nesting, which is having the expected result **always returned at the end**, and the **conditional checking performed first**. This ensures your data gets validated step by step, rather than all over the place. We also **return early**, if a condition is not met.

With each incremental step, your code will get closer to the final result. Here is how the first function would look like:

```python
def get_user_from_database(user_id: str) -> str | None:
    # Check for the user_id, otherwise return
    if user_id is None:
        return None
    # From this point on, the user_id is guaranteed to be not None.
    return User(user_id)
```

And the second function:

```python
def function(user_id: str) -> User:
    user = get_user_from_database(user_id)
    # Check result from first function, otherwise raise exception
    if not user:
        raise ValueError("User not found")
    # Check permissions, otherwise raise exception
    if not user.has_permissions():
        raise ValueError("user does not have permissions")
    # The result is validated, we can return it
    return user
```

## Automated testing

In the previous class, we have covered how to write unit tests and the concepts of test-driven development. The goal of the automated testing is to ensure all changes done are tested elsewhere, so that we don't have to always test it in our machine.

This is achieved using runners like [Github workflows](https://docs.github.com/en/actions/concepts/workflows-and-actions/workflows) or [Gitlab pipelines](https://docs.gitlab.com/ci/pipelines/).

## Development workflow

One of the most common development approaches is to use _trunk-based development_, where developers merge small, frequent updates to a trunk, often called the _main_ branch.

How to develop (an example):

- Clone the repository.
- Create a branch that introduces a feature or fixes a bug.
- Work on this branch until the feature is ready with proper testing and documentation.
- Submit the changes for review, including a proper full-text explanation of the changes.
- Once the changes are approved, they are merged into the main "trunk", ready for a future release.

<center><img src="../images/development-flow.svg" width="60%"/></center>

Read more about [Trunk-based development](https://trunkbaseddevelopment.com/).

## CI/CD

CI/CD stands for _Continuous integration, continuous delivery_, and it intends to automate most of the aspects about shipping software.

When well configured and if your processes are well defined and standardized, the entire pipeline or workflow will be executed on a remote machine to ensure that:

- Your code is valid (can be compiled, has no linting errors).
- You code follows the guidelines.
- Your tests pass (unit and integration tests).
- Often tests against best-practices and security issues using static code.
- Code is reviewed.
- Code gets released and automatically deployed.

<center><img src="../images/ci-cd-flow.webp" width="60%"/></center>

This is achieved using runners like [Github workflows](https://docs.github.com/en/actions/concepts/workflows-and-actions/workflows) or [Gitlab pipelines](https://docs.gitlab.com/ci/pipelines/). Each platform will also offer automations for software releases, as well as code reviewing and issue tracking, which this class will not explain in detail.